In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import backtrader as bt
from backtrader.sizers import PercentSizer
import backtrader.analyzers as btanalyzers

from termcolor import colored

from base import StrategyBase
import util

%matplotlib inline
%matplotlib widget

In [2]:
PATH = r'../data/stooq/data/daily/'
MARKET = r'de/xetra stocks'
TICKER = 'bayn'
FILE_CSV = PATH+MARKET+"/1/"+TICKER+".csv"

print(FILE_CSV)

../data/stooq/data/daily/de/xetra stocks/1/bayn.csv


In [3]:
class FullMoney(PercentSizer):
    params = (
        ('percents', 99),
    )

In [4]:
class BuyAndHold_1(StrategyBase):
    
    def __init__(self):
        StrategyBase.__init__(self)
        self.log("Using BuyAndHold_1 strategy")

        self.profit = 0
        
    def nextstart(self):
        self.buy()

    def stop(self):
        self.sell()

In [5]:
#Get Data
data = bt.feeds.GenericCSVData(dataname=FILE_CSV,params = (
        ('nullvalue', float('NaN')),
        ('dtformat', '%Y%m%d'),
        ('timeframe', bt.TimeFrame.Days),
        ('fromdate', datetime(2000, 1, 1)),
        ('todate', datetime(2000, 12, 31)),
        ('compression', 1),
        ('datetime', 0),
        ('time', -1),
        ('open', 1),
        ('high', 2),
        ('low', 3,
        ('close', 4),
        ('volume', 5)
        )))

cerebro = bt.Cerebro(quicknotify=True)

cerebro.adddata(data)

broker = cerebro.getbroker()
broker.setcommission(commission=0.001)  # Simulating exchange fee
broker.setcash(100000.0)
cerebro.addsizer(FullMoney)

# Analyzers to evaluate trades and strategies
# SQN = Average( profit / risk ) / StdDev( profit / risk ) x SquareRoot( number of trades )
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Years)
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharperatio')
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annual')
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(btanalyzers.DrawDown)

# Include Strategy
cerebro.addstrategy(BuyAndHold_1)

initial_value = cerebro.broker.getvalue()
print('Starting Portfolio Value: %.2f' % initial_value)
results = cerebro.run()
strat = results[0]

portfolio_stats = strat.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)

# Print analyzers - results
final_value = cerebro.broker.getvalue()
print('Final Portfolio Value: %.2f' % final_value)
print('Profit %.3f%%' % ((final_value - initial_value) / initial_value * 100))

# If no name has been specified, the name is the class name lowercased
tret_analyzer = strat.analyzers.getbyname('timereturn')
maxdrawdown = strat.analyzers.drawdown.get_analysis()['max']['drawdown']
cagr = strat.analyzers.returns.get_analysis()['rnorm100']
sharpe = strat.analyzers.sharperatio.get_analysis()['sharperatio']
print(f"Max Drawdown: {maxdrawdown:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")

Starting Portfolio Value: 100000.00
[09-04-21 15:44] Base strategy initialized
[09-04-21 15:44] Using BuyAndHold_1 strategy
[24-09-09 23:59] BUY EXECUTED, Price: 30.91, Cost: 98411.72, Comm 98.41
Final Portfolio Value: 171870.64
Profit 71.871%
Max Drawdown: 64.88%
CAGR: 4.78%
Sharpe: 0.255


In [7]:
import matplotlib
matplotlib.use('agg')

%matplotlib inline
%matplotlib widget

cerebro.plot(iplot= False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[<Figure size 432x288 with 4 Axes>]]

#### Increment the portfolio with an amount of cash every month

In [8]:
class BuyAndHold_More(bt.Strategy):
    params = dict(
        monthly_cash=10000.0,  # amount of cash to buy every month
    )

    def start(self):
        self.cash_start = self.broker.get_cash()
        self.val_start = 10000.0

        # Add a timer which will be called on the 1st trading day of the month
        self.add_timer(
            bt.timer.SESSION_END,  # when it will be called
            monthdays=[1],  # called on the 1st day of the month
            monthcarry=True,  # called on the 2nd day if the 1st is holiday
        )

    def notify_timer(self, timer, when, *args, **kwargs):
        # Add the influx of monthly cash to the broker
        self.broker.add_cash(self.p.monthly_cash)

        # buy available cash
        target_value = self.broker.get_value() + self.p.monthly_cash
        self.order_target_value(target=target_value)

    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.cash_start) - 1.0
        print('ROI:        {:.2f}%'.format(self.roi))

In [9]:
cerebro = bt.Cerebro()

idx = cerebro.addstrategy(BuyAndHold_More)
cerebro.addsizer(bt.sizers.AllInSizer)

In [10]:
cerebro.adddata(data)

# Add Analyzers
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Years)
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharperatio')
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annual')
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(btanalyzers.DrawDown)

start_portfolio_value = cerebro.broker.getvalue()
print('Starting Portfolio Value: %.2f' % start_portfolio_value)

results = cerebro.run(maxcpu=4)
strat = results[0]
portfolio_stats = strat.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)

end_portfolio_value = cerebro.broker.getvalue()
print('Final Portfolio Value: %.2f' % end_portfolio_value)
pnl = end_portfolio_value - start_portfolio_value
print('PnL: %.2f' % pnl)

# If no name has been specified, the name is the class name lowercased
tret_analyzer = strat.analyzers.getbyname('timereturn')
maxdrawdown = strat.analyzers.drawdown.get_analysis()['max']['drawdown']
print(f"Max Drawdown: {maxdrawdown:.2f}")


Starting Portfolio Value: 10000.00
ROI:        135.60%
Final Portfolio Value: 1366022.00
PnL: 1356022.00
Max Drawdown: 49.22


In [12]:
# Create a Data Feed from investpy
PATH = "../data/investpy/data/index/"
data_sp = bt.feeds.GenericCSVData(
    dataname=PATH+'S&P_500.csv',

    fromdate=datetime(2000, 1, 1),
    todate=datetime(2020, 12, 31),

    nullvalue=0.0,

    dtformat=('%Y-%m-%d'),

    datetime=0,
    high=1,
    low=2,
    open=3,
    close=4,
    volume=5,
#     openinterest=-1
)